In [46]:
import os
import re
import sys
import string
import numpy as np
import pandas as pd
import gensim, logging
from pymystem3 import Mystem

In [41]:
path = './weka/weka/'
classes = ['anekdots', 'izvest', 'teh_mol']
datas = []

for cls in classes:
    pth = os.path.join(path, cls)
    for i in os.listdir(pth):
        with open(os.path.join(pth, i)) as f:
            datas.append([cls, f.read()])

In [62]:
data = pd.DataFrame(datas, columns=['cls', 'text'])

In [43]:
data.tail()

,cls,text
370,teh_mol,"Д. Арнаудов. По просьбам репортеров\nСнимки, н..."
371,teh_mol,"А. Соломин, В. Соломин. Непростые фигурные чис..."
372,teh_mol,Е. Блохин. На вас вся надежда\nЯ недавно побыв...
373,teh_mol,"Олег Сагоян, Владимир Диденко. Гимнастический ..."
374,teh_mol,Анатолий Карташкин. Устройства для мистификаци...


In [63]:
# здесь должны были убираться знаки препинания, ноо нет
m = Mystem()
data['text'] = data['text'].map(lambda x: list(map(lambda y: m.lemmatize(y)[0], x.split())))
data.head()

,cls,text
0,anekdots,"[абстрактный, анекдот, ***\n, лететь, два, кир..."
1,anekdots,"[андропов, ***\n, --\n, какой, поэзия, любить,..."
2,anekdots,"[англичанин, ***\n, пожилой, джентльмен, приуч..."
3,anekdots,"[аптекарь, ***\n, старый, аптекарь, оставлять,..."
4,anekdots,"[военный, ***\n, в, армия, нет, вечный, двигат..."


In [45]:
mod = 'ruscorpora_1_300_10.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(mod, binary=True)

In [69]:
tags_my = ['S', 'A', 'ADV', 'V']
tags_mo = ['NOUN', 'ADJ', 'ADV', 'VERB']
tags = {tags_my[i]: tags_mo[i] for i in range(len(tags_my))}

In [90]:
vectors = []
i = 0
for txt in data['text']:
    raw_v = []
    for wd in txt:
        try:
            wt = m.analyze(wd)[0]['analysis'][0]['gr'].split(',')[0].split('=')[0]
            raw_v.append(model['{}_{}'.format(wd, tags[wt])])
        except:
            continue
    vectors.append(np.mean(np.array(raw_v), axis=0))

In [91]:
len(data['text'])==len(vectors)

True

Часть вторая

In [94]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
 
clf = LogisticRegression(penalty="l2", solver="lbfgs", multi_class="multinomial", max_iter=300, n_jobs=4)

In [93]:
X_train, X_test, y_train, y_test = train_test_split(vectors, data['cls'])

In [ ]:
cross_val_predict(clf, X_train, y_train)